# Realtime Miniで構築する

子供の頃、私はJarvis（複雑なワークフローを自律的に処理できるインテリジェントアシスタント）のアイデアに魅了されていました。当時は気づいていませんでしたが、私は音声エージェントの未来を想像していたのです。OpenAIは`4o-audio`のリリースでこのビジョンを最初に実現し、最近では[GPT Realtime Mini](https://platform.openai.com/docs/models/gpt-realtime-mini)のリリースによって、コストを70%削減し、より低いレイテンシとツール呼び出しの大幅な改善を提供することで、さらにアクセスしやすくしました。

しかし、音声モデルでの構築は、テキストのみのインターフェースでの作業とは根本的に異なります。プロンプトエンジニアリングに加えて、音声モデルには新たな課題があります：レイテンシにより敏感で、WebRTCセッションの管理が必要で、音声活動検出（VAD）による追加の変動性が導入されます。

このプロセスを簡単にするため、OpenAIはPythonとTypeScriptの両方でAgents SDKをリリースし、信頼性の高い音声エージェントを構築するための推奨設計パターンを示す詳細な例も提供しています。

コードに入る前に、私たちが構築しようとしているものを正確にマッピングし、それがより広範なエージェントハンドオフアーキテクチャにどのように適合するかを見てみましょう。

## システムアーキテクチャ
今日構築するアプリケーションでは、**「ハンドオフアーキテクチャ」**を使用して非常にシンプルなカスタマーサポートアプリを構築します。**「ハンドオフアーキテクチャ」**とは、**プライマリエージェント**がすべての顧客からの問い合わせに対するオーケストレーターとして機能することを意味します。すべてのリクエストを直接処理するのではなく、プライマリエージェントはユーザーのメッセージの背後にある意図を分析し、**2つの主要なパスウェイのいずれかに分類します**：

1. 一般的な質問と基本的なサポート（認証機能は不要）
2. 具体的な質問（検索を実行する前にユーザー認証が必要）

この分類に基づいて、プライマリエージェントは**会話を引き継いで**、その特定のタスクのために設計された適切なスペシャリストエージェントに渡します。

![alt text](../../images/byo_realtime_diagram.png)

## セットアップ
ゼロから始める代わりに、[openai-agents-js](https://github.com/openai/openai-agents-js/tree/main) リポジトリを使って作業を進めていきます。まずはクローンして、必要な依存関係をインストールし、Webデモをビルドしましょう。

```bash
git clone https://github.com/openai/openai-agents-js/tree/main
```

クローン後、readmeの手順に従って開始してください。

```bash
npm install @openai/agents zod@3
pnpm examples:realtime-next
```

すべてが期待通りに動作すれば、シンプルなチャットインターフェースが表示されるはずです。
![alt text](../../images/byo_realtime_starting.png)

## メインエージェント
素晴らしい！リポジトリをクローンしたので、`openai-agents-js/examples/realtime-next/src/app/page.tsx`を修正していきます。まずは**メインエージェント**から始めます。**メインエージェント**は、アプリケーションスタックのエントリーポイントです。ユーザーのクエリに対するインテント分類器として機能し、異なるレイヤー間でのルーティング方法を選択します。

実装は非常にシンプルです：
```js
const mainAgent = new RealtimeAgent({
  name: 'Main Agent',
  instructions:
    'You are the entry point for all customer queries. Default to the no-auth QA flow. If authentication is needed and validated, escalate to the Auth Layer by handing off to either the Flight Status Checker or Rebooking Agent. Do not answer policy questions from your own knowledge; rely on subordinate agents and tools.',
  tools: [
    checkFlightsTool,
  ],
  handoffs: [qaAgent],
});
```

## QAエージェント

メインエージェントを構築したので、次のステップは特定のクラスの顧客クエリを処理する専門的なサポートエージェントを追加することです。一般的な航空会社のポリシーに関する質問については、これがQAエージェントになります。

実際の製品では、このエージェントはより洗練された体験を提供します：会社固有のPDFやその他の参考資料を取り込み、それらを埋め込み、実行時にそれらのドキュメントを動的にクエリして、正確でポリシーに基づいた回答を提供します。

```
┌────────────┐      ┌────────────┐      ┌────────────────────────┐      ┌────────────┐
│ User Query │ ───► │ QA Agent   │ ───► │ Vector DB / Retriever  │ ───► │ LLM Answer │
└────────────┘      └────────────┘      └────────────────────────┘      └────────────┘
                          │                     │
                          │ build search        │ top-k context
                          ▼                     ▼
                 (semantic search)      (grounded generation)

```

これは通常、顧客のクエリを埋め込み、最も関連性の高い結果を取得する完全なベクターデータベースサービスの構築を含みます。このデモでは簡単にするために、パイプラインのその部分をモックします。

完全機能の検索システムの実装方法に興味がある場合は、このトピックに関する他のクックブック[こちら](https://cookbook.openai.com/examples/vector_databases/pinecone/readme)をご覧ください。

```js
const documentLookupTool = tool({
  name: 'document_lookup_tool',
  description: 'Looks up answers from known airline documentation to handle general questions without authentication.',
  parameters: z.object({
    request: z.string(),
  }),
  execute: async ({ request }) => {
    const mockDocument = `**Airline Customer Support — Quick Reference**

1. Each passenger may bring 1 carry-on (22 x 14 x 9) and 1 personal item.
2. Checked bags must be under 50 lbs; overweight fees apply.
3. Online check-in opens 24 hours before departure.
4. Seat upgrades can be requested up to 1 hour before boarding.
5. Wi‑Fi is complimentary on all flights over 2 hours.
6. Customers can change flights once for free within 24 hours of booking.
7. Exit rows offer extra legroom and require passengers to meet safety criteria.
8. Refunds can be requested for canceled or delayed flights exceeding 3 hours.
9. Pets are allowed in the cabin if under 20 lbs and in an approved carrier.
10. For additional help, contact our support team via chat or call center.`;
    return mockDocument;
  },
});
```

以前にメインエージェントを定義したときと同様に、`RealtimeAgent`の別のインスタンスを作成しますが、今回は`documentLookupTool`を提供します。

```js
const qaAgent = new RealtimeAgent({
  name: 'QA Agent',
  instructions:
    'You handle general customer questions using the document lookup tool. Use only the document lookup for answers. If the request may involve personal data or operations (rebooking, flight status), call the auth check tool. If auth is required and validated, handoff to the appropriate Auth Layer agent.',
  tools: [documentLookupTool],
});
```

## フライト状況エージェント
私たちはすでに強力な基盤を構築しました：顧客からの問い合わせを処理できるメインエージェントと、ドキュメントストアを検索して正確でポリシーに基づく回答を提供するQAエージェントです。

不足しているのは顧客固有の情報を扱う層です。例えば、「私のフライトの状況はどうですか？」や「どのターミナルに行けばよいですか？」といった問い合わせです。このような個人化されたやり取りをサポートするには、システムがユーザー固有のデータに安全にアクセスして応答できるよう、ワークフローに認証層を組み込む必要があります。

```
┌────────────┐      ┌──────────────┐      ┌───────────────────────┐      ┌───────────────────────┐
│ User Query │ ───► │ Auth Layer   │ ───► │ Customer Data Access  │ ───► │ LLM Answer (Personal) │
└────────────┘      └──────────────┘      └───────────────────────┘      └───────────────────────┘
                        │                          │
                        │ verify identity          │ query flight / account
                        ▼                          ▼
                (token, SSO, OTP, etc.)   (e.g., flight status, profile info)
```
幸い、Agents SDKはこのような使用例をサポートするよう設計されています。機密性の高いアカウントレベルの情報を含む顧客サポートシナリオでは、`tool`内の`needsApproval`パラメータを使用することで適切なアクセス制御を確保できます。これにより、保護されたデータにアクセスする前にユーザーの認証が必要になります。

```js
const checkFlightsTool = tool({
  name: 'checkFlightsTool',
  description: 'Call this tool if the user queries about their current flight status',
  parameters: z.object({}),
  // Require approval so the UI can collect creds before executing.
  needsApproval: true,
  execute: async () => {
    if (!credState.username || !credState.password) {
      return 'Authentication missing.';
    }
    return `${credState.username} you are currently booked on the 8am flight from SFO to JFK`;
  },
});
```

ツールが`needsApproval`で登録されると、セッション中に自動的に`tool_approval_requested`イベントが発行されます。これにより、Webアプリケーションの`RealtimeAgent`インスタンス化ブロック内にロジックを追加して、これらのイベントをリッスンし、それに応じてUIを更新することができます。例えば、続行する前にユーザーに承認や認証を求めるプロンプトを表示することができます。

```js
  const [credUsername, setCredUsername] = useState('');
  const [credPassword, setCredPassword] = useState('');
  const [pendingApproval, setPendingApproval] = useState<any | null>(null);

  useEffect(() => {
    session.current = new RealtimeSession(mainAgent, {
        // other configs go here! 
    });
    // various other event based logic goes here!
    session.current.on(
      'tool_approval_requested',
      (_context, _agent, approvalRequest) => {
        setPendingApproval(approvalRequest.approvalItem); // <- Alterations to react state!
        setCredUsername('');
        setCredPassword('');
        setCredOpen(true);
      },
    );
  }, []);
  // ....
  return (
    {credOpen && (
        <div className="fixed inset-0 z-50">
          // ... remainder of component logic
        </div>
      )}
  )
```

## 最終的なコードスニペット
これで完了です！これで顧客サポートアプリケーションのコアコンポーネントを構築できました：

* 幅広い顧客サポートクエリを処理できる汎用エージェント
* ユーザーの身元を確認し、顧客固有の情報を取得する認証ワークフロー

すべてが整ったので、`realtime-next/src/app/page.tsx`の最終バージョンは以下のようになります。

```js
'use client';

import {
  RealtimeAgent,
  RealtimeSession,
  tool,
  TransportEvent,
  RealtimeOutputGuardrail,
  OutputGuardrailTripwireTriggered,
  RealtimeItem,
} from '@openai/agents/realtime';
import { useEffect, useRef, useState } from 'react';
import { z } from 'zod';
import { getToken } from './server/token.action';
import { App } from '@/components/App';
import { CameraCapture } from '@/components/CameraCapture';

// ツールが実行時に読み取れるデモ専用の認証情報ストア
const credState: { username?: string; password?: string } = {};

// ---------------------------------------------
// ツール

const documentLookupTool = tool({
  name: 'document_lookup_tool',
  description: '認証なしで一般的な質問を処理するために、既知の航空会社ドキュメントから回答を検索します。',
  parameters: z.object({
    request: z.string(),
  }),
  execute: async ({ request }) => {
    const mockDocument = `**航空会社カスタマーサポート — クイックリファレンス**

1. 各乗客は機内持ち込み手荷物1個（22 x 14 x 9）と身の回り品1個を持参できます。
2. 預け手荷物は50ポンド未満である必要があります；超過重量料金が適用されます。
3. オンラインチェックインは出発24時間前に開始されます。
4. 座席のアップグレードは搭乗1時間前まで要求できます。
5. 2時間を超えるすべてのフライトでWi‑Fiは無料です。
6. 顧客は予約から24時間以内に1回無料でフライトを変更できます。
7. 非常口座席は足元が広く、乗客は安全基準を満たす必要があります。
8. 3時間を超えるキャンセルまたは遅延フライトについては返金を要求できます。
9. 20ポンド未満で承認されたキャリアに入れられている場合、ペットは客室に持ち込めます。
10. 追加のヘルプについては、チャットまたはコールセンター経由でサポートチームにお問い合わせください。`;
    return mockDocument;
  },
});

const checkFlightsTool = tool({
  name: 'checkFlightsTool',
  description: 'ユーザーが現在のフライト状況について問い合わせた場合にこのツールを呼び出します',
  parameters: z.object({}),
  // 実行前にUIが認証情報を収集できるように承認を要求します。
  needsApproval: true,
  execute: async () => {
    if (!credState.username || !credState.password) {
      return '認証が不足しています。';
    }
    return `${credState.username}様、現在SFOからJFKへの午前8時のフライトをご予約いただいています`;
  },
});

// ---------------------------------------------
// 各レイヤーのエージェント

// 2. 認証なしレイヤー：ドキュメント検索と認証チェックツールを持つQAエージェント
const qaAgent = new RealtimeAgent({
  name: 'QA Agent',
  instructions:
    'ドキュメント検索ツールを使用して一般的な顧客の質問を処理します。回答にはドキュメント検索のみを使用してください。リクエストが個人データや操作（再予約、フライト状況）に関わる可能性がある場合は、認証チェックツールを呼び出してください。認証が必要で検証された場合は、適切な認証レイヤーエージェントに引き継いでください。',
  tools: [documentLookupTool],
});

// 1. メインエージェント：エントリーポイントとルーティング
const mainAgent = new RealtimeAgent({
  name: 'Main Agent',
  instructions:
    'すべての顧客クエリのエントリーポイントです。デフォルトでは認証なしのQAフローを使用してください。認証が必要で検証された場合は、フライト状況チェッカーまたは再予約エージェントのいずれかに引き継いで認証レイヤーにエスカレートしてください。自分の知識からポリシーの質問に答えないでください；下位エージェントとツールに依存してください。',
  tools: [
    checkFlightsTool,
  ],
  handoffs: [qaAgent],
});

// エージェントが戻ったりエスカレートしたりできるようにクロス引き継ぎ
qaAgent.handoffs = [mainAgent];

export default function Home() {
  const session = useRef<RealtimeSession<any> | null>(null);
  const [isConnected, setIsConnected] = useState(false);
  const [isMuted, setIsMuted] = useState(false);
  const [outputGuardrailResult, setOutputGuardrailResult] =
    useState<OutputGuardrailTripwireTriggered<any> | null>(null);

  const [events, setEvents] = useState<TransportEvent[]>([]);
  const [history, setHistory] = useState<RealtimeItem[]>([]);
  const [mcpTools, setMcpTools] = useState<string[]>([]);
  const [credOpen, setCredOpen] = useState(false);
  const [credUsername, setCredUsername] = useState('');
  const [credPassword, setCredPassword] = useState('');
  const [pendingApproval, setPendingApproval] = useState<any | null>(null);

  useEffect(() => {
    session.current = new RealtimeSession(mainAgent, {
      model: 'gpt-realtime-mini',
      outputGuardrailSettings: {
        debounceTextLength: 200,
      },
      config: {
        audio: {
          output: {
            voice: 'cedar',
          },
        },
      },
    });
    session.current.on('transport_event', (event) => {
      setEvents((events) => [...events, event]);
    });
    session.current.on('mcp_tools_changed', (tools) => {
      setMcpTools(tools.map((t) => t.name));
    });
    session.current.on(
      'guardrail_tripped',
      (_context, _agent, guardrailError) => {
        setOutputGuardrailResult(guardrailError);
      },
    );
    session.current.on('history_updated', (history) => {
      setHistory(history);
    });
    session.current.on(
      'tool_approval_requested',
      (_context, _agent, approvalRequest) => {
        setPendingApproval(approvalRequest.approvalItem);
        setCredUsername('');
        setCredPassword('');
        setCredOpen(true);
      },
    );
  }, []);

  async function connect() {
    if (isConnected) {
      await session.current?.close();
      setIsConnected(false);
    } else {
      const token = await getToken();
      try {
        await session.current?.connect({
          apiKey: token,
        });
        setIsConnected(true);
      } catch (error) {
        console.error('セッションへの接続エラー', error);
      }
    }
  }

  async function toggleMute() {
    if (isMuted) {
      await session.current?.mute(false);
      setIsMuted(false);
    } else {
      await session.current?.mute(true);
      setIsMuted(true);
    }
  }

  function handleCredCancel() {
    const approval = pendingApproval;
    setCredOpen(false);
    setPendingApproval(null);
    if (approval) session.current?.reject(approval);
  }

  function handleCredSubmit(e: React.FormEvent) {
    e.preventDefault();
    if (!credUsername || !credPassword) return;
    // ツールが読み取れるように認証情報を保存
    credState.username = credUsername;
    credState.password = credPassword;
    const approval = pendingApproval;
    setCredOpen(false);
    setPendingApproval(null);
    setCredUsername('');
    setCredPassword('');
    if (approval) session.current?.approve(approval);
  }

  return (
    <div className="relative">
      {credOpen && (
        <div className="fixed inset-0 z-50">
          <div className="absolute inset-0 bg-black/50" />
          <div className="fixed top-0 left-0 right-0 flex justify-center p-4">
            <form
              onSubmit={handleCredSubmit}
              className="w-full max-w-sm rounded-lg bg-white p-4 shadow-xl"
            >
              <div className="mb-2 text-sm font-semibold">認証が必要です</div>
              <div className="mb-3 text-xs text-gray-600">
                続行するにはユーザー名とパスワードを入力してください。
              </div>
              <input
                className="mb-2 w-full rounded border border-gray-300 p-2 text-sm focus:border-gray-500 focus:outline-none"
                placeholder="ユーザー名"
                value={credUsername}
                onChange={(e) => setCredUsername(e.target.value)}
              />
              <input
                type="password"
                className="mb-3 w-full rounded border border-gray-300 p-2 text-sm focus:border-gray-500 focus:outline-none"
                placeholder="パスワード"
                value={credPassword}
                onChange={(e) => setCredPassword(e.target.value)}
              />
              <div className="flex justify-end gap-2">
                <button
                  type="button"
                  className="rounded bg-gray-100 px-3 py-1.5 text-sm hover:bg-gray-200"
                  onClick={handleCredCancel}
                >
                  キャンセル
                </button>
                <button
                  type="submit"
                  className="rounded bg-black px-3 py-1.5 text-sm text-white hover:bg-gray-800 disabled:opacity-50"
                  disabled={!credUsername || !credPassword}
                >
                  続行
                </button>
              </div>
            </form>
          </div>
        </div>
      )}
      <App
        isConnected={isConnected}
        isMuted={isMuted}
        toggleMute={toggleMute}
        connect={connect}
        history={history}
        outputGuardrailResult={outputGuardrailResult}
        events={events}
        mcpTools={mcpTools}
      />
      <div className="fixed bottom-4 right-4 z-50">
        <CameraCapture
          disabled={!isConnected}
          onCapture={(dataUrl) => {
            if (!session.current) return;
            session.current.addImage(dataUrl, { triggerResponse: false });
          }}
        />
      </div>
    </div>
  );
}
```